# Feedback Amplifier

> Refs:
> * _Introduction to Radio Frequency Design_, Wes Hayward [Chapter 5 Two-Port Networks]
> * _Experimental Methods in RF Design_, w7zoi, kk7b, w7pua - CDROM

In [1]:
import sys

sys.path.append("../../lib")
from feedback import interactive_complex_fba

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

display(interactive_complex_fba)

interactive(children=(FloatText(value=10.0, description='$F$ MHz', style=DescriptionStyle(description_width='2…